# Cleaning for Google Trends

## 1. Load packages and data

In [1]:
# import packages
import os 
import pickle

import re
from datetime import datetime

import requests

import pytz

import pandas as pd
import numpy as np

import ast

import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import array_contains


import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px

import emojis


In [2]:
# import findspark
import findspark

# initialize findspark with spark directory

#ALWAYS HAVE TO BE CHANGED 
findspark.init("/Users/wouterdewitte/spark/")

# import pyspark
import pyspark
# create spark context
sc = pyspark.SparkContext()
# create spark session 
spark = pyspark.sql.SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/26 11:37:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/26 11:37:35 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
#set this path to your path, for some reason I have an error 
#reading in all the files
import os
path_json = ".././../data/Topic_vegan/*.json"

In [4]:
df_json = spark.read.json(path_json)

22/11/26 11:38:42 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
# select interesting features
import pyspark.sql.functions as F

df = df_json.select(F.col("user.name"),
                    F.col("user.screen_name"),
                    F.col("created_at"), 
                    F.col("full_text"),
                    F.col("user.followers_count"),
                    F.col("retweet_count"),
                    F.col("favorite_count"),
                    F.col("entities.hashtags"))
df.limit(10).toPandas()

,name,screen_name,created_at,full_text,followers_count,retweet_count,favorite_count,hashtags
0,のり/Nori,nori_k_629,Mon Apr 04 10:09:55 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,139,3582,0,[]
1,alice,myn4meizalize,Mon Apr 04 10:09:54 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...,655,3837,0,[]
2,Karen Reed 🌸,kandk670,Mon Apr 04 10:09:54 +0000 2022,@trudiebakescake Organic coconut oil in a jar ...,711,0,0,[]
3,ハル):),patlnwza55,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,236,3582,0,[]
4,alice,myn4meizalize,Mon Apr 04 10:09:52 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,655,3582,0,[]
5,ﾌｧﾙﾄﾞﾗっ子organic有機💙💻,organic_yusai,Mon Apr 04 10:09:52 +0000 2022,マジでピンチ助けて自転車ガガガガ,291,0,1,[]
6,のり/Nori,nori_k_629,Mon Apr 04 10:09:50 +0000 2022,RT @mynameisnanon: คิดถึงกันป่าว ถ้าคิดถึงต้อง...,139,3837,0,[]
7,｡◕‿◕｡𝑱𝒆 𝒕'𝒂𝒊𝒎𝒆 🐶🧡✨,MyFnlovely97,Mon Apr 04 10:09:50 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,245,3582,0,[]
8,Sang™,asan_gk,Mon Apr 04 10:09:50 +0000 2022,RT @NotechAna: Am I the only one who types in ...,2065,374,0,[]
9,Trysia ):)▪︎never let me go▪︎,Winnie_thephuu,Mon Apr 04 10:09:48 +0000 2022,RT @ohmpawatt: เพื่อนๆคิดถึงผมมั้ยยย ถ้าคิดถึง...,379,3582,0,[]


## 2. Preprocess Data

In [9]:
# https://developer.twitter.com/en/docs/twitter-ads-api/timezones
# function to convert Twitter date string format
def getDate(date):
    if date is not None:
        return str(datetime.strptime(date,'%a %b %d %H:%M:%S +0000 %Y').replace(tzinfo=pytz.UTC).strftime("%Y-%m-%d %H:%M:%S"))
    else:
        return None

# UDF declaration
date_udf = F.udf(getDate, StringType())

# apply udf
df = df.withColumn('post_created_at', F.to_utc_timestamp(date_udf("created_at"), "UTC"))

In [10]:
df.show()

22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_23 in memory! (computed 3.4 MiB so far)
22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_18 in memory! (computed 3.4 MiB so far)


22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_29 in memory! (computed 3.5 MiB so far)
22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_31 in memory! (computed 3.5 MiB so far)
22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_32 in memory! (computed 3.4 MiB so far)
22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_34 in memory! (computed 3.4 MiB so far)


22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_39 in memory! (computed 3.5 MiB so far)
22/11/26 11:40:58 WARN MemoryStore: Not enough space to cache rdd_30_40 in memory! (computed 3.5 MiB so far)


22/11/26 11:40:59 WARN MemoryStore: Not enough space to cache rdd_30_43 in memory! (computed 3.5 MiB so far)
22/11/26 11:40:59 WARN MemoryStore: Not enough space to cache rdd_30_48 in memory! (computed 3.4 MiB so far)


22/11/26 11:40:59 WARN MemoryStore: Not enough space to cache rdd_30_53 in memory! (computed 3.4 MiB so far)
22/11/26 11:40:59 WARN MemoryStore: Not enough space to cache rdd_30_55 in memory! (computed 3.5 MiB so far)
22/11/26 11:40:59 WARN MemoryStore: Not enough space to cache rdd_30_54 in memory! (computed 3.4 MiB so far)
22/11/26 11:40:59 WARN MemoryStore: Not enough space to cache rdd_30_59 in memory! (computed 3.3 MiB so far)


...

22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_160 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_159 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_158 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_162 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_161 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_165 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:03 WARN MemoryStore: Not enough space to cache rdd_30_168 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_171 in memory! (computed 3.5 MiB so far)


22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_175 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_177 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_176 in memory! (computed 3.3 MiB so far)


22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_181 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_183 in memory! (computed 3.3 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_186 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_185 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_187 in memory! (computed 3.4 MiB so far)


22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_194 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_195 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:04 WARN MemoryStore: Not enough space to cache rdd_30_197 in memory! (computed 3.4 MiB so far)


+--------------------+---------------+--------------------+--------------------+---------------+-------------+--------------+--------------------+-------------------+
|                name|    screen_name|          created_at|           full_text|followers_count|retweet_count|favorite_count|            hashtags|    post_created_at|
+--------------------+---------------+--------------------+--------------------+---------------+-------------+--------------+--------------------+-------------------+
| Follow the Vegans Ⓥ|  vegan_v_vegan|Sat May 14 00:55:...|!\n#vegan #GoVega...|           4285|            1|             6|[{[2, 8], vegan},...|2022-05-14 00:55:33|
|🌱Veg-In-Out Mark...| veginoutmarket|Sat Jan 15 07:17:...|! We will be open...|            947|            0|             0|[{[69, 79], vegan...|2022-01-15 07:17:18|
|         Mix 93.8 FM|       Mix938FM|Wed Sep 07 09:11:...|!! Daily Updates ...|          10745|            3|             8|[{[52, 67], TheMo...|2022-09-07 09:11:40|

In [11]:
#drop duplicates and retweets 
df = df.filter(~F.col("full_text").startswith("RT"))\
                        .drop_duplicates().cache()
#sorting such when dropping later we only keep the most recent post 
#df = df.sort("post_created_at", ascending=False)
#removing spam accounts 
df = df.drop_duplicates(["full_text", "screen_name"])

df.show()             
#df.printSchema()
#df.count() #1340938

22/11/26 11:41:07 WARN MemoryStore: Not enough space to cache rdd_30_29 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:07 WARN MemoryStore: Not enough space to cache rdd_30_32 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:07 WARN MemoryStore: Not enough space to cache rdd_30_33 in memory! (computed 3.5 MiB so far)


22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_39 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_41 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_40 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_42 in memory! (computed 3.4 MiB so far)


22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_49 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_50 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_47 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_53 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_54 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_52 in memory! (computed 3.4 MiB so far)


22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_57 in memory! (computed 3.3 MiB so far)
22/11/26 11:41:08 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_30_62 in memory.
22/11/26 11:41:08 WARN MemoryStore: Not enough space to cache rdd_30_62 in memory! (computed 384.0 B so far)


22/11/26 11:41:09 WARN MemoryStore: Not enough space to cache rdd_30_63 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:09 WARN MemoryStore: Not enough space to cache rdd_30_71 in memory! (computed 3.4 MiB so far)
22/11/26 11:41:09 WARN MemoryStore: Not enough space to cache rdd_30_69 in memory! (computed 3.5 MiB so far)
22/11/26 11:41:09 WARN MemoryStore: Not enough space to cache rdd_30_72 in memory! (computed 3.5 MiB so far)
[215.492s][warning][gc,alloc] Executor task launch worker for task 73.0 in stage 16.0 (TID 8345): Retried waiting for GCLocker too often allocating 7219 words
[215.495s][warning][gc,alloc] Executor task launch worker for task 70.0 in stage 16.0 (TID 8342): Retried waiting for GCLocker too often allocating 256 words
[215.495s][warning][gc,alloc] Executor task launch worker for task 71.0 in stage 16.0 (TID 8343): Retried waiting for GCLocker too often allocating 4098 words
[215.504s][warning][gc,alloc] Executor task launch worker for task 72.0 in stage 16.0 (T

Py4JJavaError: An error occurred while calling o95.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 71 in stage 16.0 failed 1 times, most recent failure: Lost task 71.0 in stage 16.0 (TID 8343) (192.168.1.13 executor driver): java.lang.OutOfMemoryError: Java heap space
	at net.jpountz.lz4.LZ4BlockOutputStream.<init>(LZ4BlockOutputStream.java:102)
	at org.apache.spark.io.LZ4CompressionCodec.compressedOutputStream(CompressionCodec.scala:144)
	at org.apache.spark.serializer.SerializerManager.wrapForCompression(SerializerManager.scala:159)
	at org.apache.spark.serializer.SerializerManager.wrapStream(SerializerManager.scala:134)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:163)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWritePro

In [ ]:
# define function to count hashtags
def get_hashtags(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "#" in word:
            counter += 1
    return(counter) 

# define function to count mentions
def get_mentions(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "@" in word:
            counter += 1
    return(counter)

# define function to count exclamation marks
def get_exclamation_marks(tokenized_text):
    counter = 0
    for word in tokenized_text:
        if "!" in word:
            counter += 1
    return(counter)

# define function to count number of emojis used
import emojis
def emoji_counter(text):
    nr_emojis = emojis.count(text)
    return(nr_emojis)
# register functions as udf
get_hashtags_UDF = F.udf(get_hashtags, IntegerType())
get_mentions_UDF = F.udf(get_mentions, IntegerType())
get_exclamation_marks_UDF = F.udf(get_exclamation_marks, IntegerType())
emoji_counter_udf = F.udf(emoji_counter, IntegerType())


In [ ]:
twitter_df = df.withColumn("emoji_count", emoji_counter_udf("full_text")) \
                            .withColumn("text_tokenized", F.split("full_text", " ")) \
                            .withColumn("num_words", F.size("text_tokenized")) \
                            .withColumn("num_hashtags", get_hashtags_UDF("text_tokenized")) \
                            .withColumn("num_mentions", get_mentions_UDF("text_tokenized")) \
                            .withColumn("num_exclamation_marks", get_exclamation_marks_UDF("text_tokenized"))
twitter_df.show()

In [ ]:
# define function to clean text
def clean_text(string):
    
    # define numbers
    NUMBERS = '0123456789'
    PUNCT = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    
    # convert text to lower case
    cleaned_string = string.lower()
    
    # remove URLS
    cleaned_string = re.sub(r'http\S+', ' ', cleaned_string)
    
    # replace emojis by words
    cleaned_string = emoji.demojize(cleaned_string)
    cleaned_string = cleaned_string.replace(":"," ").replace("_"," ")
    cleaned_string = ' '.join(cleaned_string.split())
    
    # remove numbers
    cleaned_string = "".join([char for char in cleaned_string if char not in NUMBERS])
    
    # remove punctuation
    cleaned_string = "".join([char for char in cleaned_string if char not in PUNCT])
    
    # remove words conisting of one character (or less)
    cleaned_string = ' '.join([w for w in cleaned_string.split() if len(w) > 1])
    
    # return
    return(cleaned_string) 
clean_text_udf = F.udf(clean_text, StringType())


In [ ]:
twitter_df = df.withColumn("cleaned_text", clean_text_udf(F.col("full_text")))

In [ ]:
twitter_df.show()

22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_127 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_124 in memory! (computed 3.3 MiB so far)
22/11/26 11:13:01 WARN BlockManager: Persisting block rdd_24_127 to disk instead.
22/11/26 11:13:01 WARN BlockManager: Persisting block rdd_24_124 to disk instead.
22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_127 in memory! (computed 3.4 MiB so far)


22/11/26 11:13:01 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_138 in memory.
22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_131 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:01 WARN BlockManager: Persisting block rdd_24_131 to disk instead.
22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_133 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:01 WARN BlockManager: Persisting block rdd_24_133 to disk instead.
22/11/26 11:13:01 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_140 in memory.
22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_134 in memory! (computed 3.3 MiB so far)
22/11/26 11:13:01 WARN BlockManager: Persisting block rdd_24_134 to disk instead.
22/11/26 11:13:01 WARN MemoryStore: Not enough space to cache rdd_24_134 in memory! (computed 3.3 MiB so far)
22/11/26 11:13:01 WARN MemoryStore: Not enou

22/11/26 11:13:02 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_143 in memory.
22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_140 in memory! (computed 384.0 B so far)
22/11/26 11:13:02 WARN BlockManager: Persisting block rdd_24_140 to disk instead.
22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_143 in memory! (computed 384.0 B so far)
22/11/26 11:13:02 WARN BlockManager: Persisting block rdd_24_143 to disk instead.
22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_137 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:02 WARN BlockManager: Persisting block rdd_24_137 to disk instead.


22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_138 in memory! (computed 3.3 MiB so far)
22/11/26 11:13:02 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_148 in memory.
22/11/26 11:13:02 WARN MemoryStore: Failed to reserve initial memory threshold of 1024.0 KiB for computing block rdd_24_151 in memory.
22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_145 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:02 WARN BlockManager: Persisting block rdd_24_145 to disk instead.


22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_148 in memory! (computed 384.0 B so far)
22/11/26 11:13:02 WARN BlockManager: Persisting block rdd_24_148 to disk instead.
22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_151 in memory! (computed 384.0 B so far)
22/11/26 11:13:02 WARN BlockManager: Persisting block rdd_24_151 to disk instead.
22/11/26 11:13:02 WARN MemoryStore: Not enough space to cache rdd_24_145 in memory! (computed 3.4 MiB so far)


...

22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_133 in memory! (computed 3.4 MiB so far)


22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_138 in memory! (computed 3.3 MiB so far)
22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_145 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_148 in memory! (computed 3.5 MiB so far)


22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_153 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_155 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_157 in memory! (computed 3.4 MiB so far)
22/11/26 11:13:09 WARN MemoryStore: Not enough space to cache rdd_24_154 in memory! (computed 3.4 MiB so far)
[184.817s][warning][gc,alloc] Executor task launch worker for task 151.0 in stage 7.0 (TID 8220): Retried waiting for GCLocker too often allocating 4098 words


22/11/26 11:13:10 ERROR Executor: Exception in task 151.0 in stage 7.0 (TID 8220)
java.lang.OutOfMemoryError: Java heap space
	at java.base/java.io.BufferedOutputStream.<init>(BufferedOutputStream.java:75)
	at org.apache.spark.storage.DiskBlockObjectWriter$ManualCloseBufferedOutputStream$1.<init>(DiskBlockObjectWriter.scala:149)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:151)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:52)
	at org.apache.spark.scheduler.Task.run(

Py4JJavaError: An error occurred while calling o160.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 151 in stage 7.0 failed 1 times, most recent failure: Lost task 151.0 in stage 7.0 (TID 8220) (192.168.1.13 executor driver): java.lang.OutOfMemoryError: Java heap space
	at java.base/java.io.BufferedOutputStream.<init>(BufferedOutputStream.java:75)
	at org.apache.spark.storage.DiskBlockObjectWriter$ManualCloseBufferedOutputStream$1.<init>(DiskBlockObjectWriter.scala:149)
	at org.apache.spark.storage.DiskBlockObjectWriter.initialize(DiskBlockObjectWriter.scala:151)
	at org.apache.spark.storage.DiskBlockObjectWriter.open(DiskBlockObjectWriter.scala:159)
	at org.apache.spark.storage.DiskBlockObjectWriter.write(DiskBlockObjectWriter.scala:306)
	at org.apache.spark.shuffle.sort.BypassMergeSortShuffleWriter.write(BypassMergeSortShuffleWriter.java:171)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.s